In [2]:
import polars as pl
import constants
import psycopg
from riotwatcher import LolWatcher, ApiError, RiotWatcher

# sys.stdout.reconfigure(encoding='utf-8')

API_KEY = constants.API_KEY_SERVICE
lol_watcher = LolWatcher(API_KEY)
riot_watcher = RiotWatcher(API_KEY)
lol_region = 'na1'

db_pass = constants.db_password
db_ip = constants.db_ip

db_connection = f'dbname = yrden user=postgres password={db_pass} host={db_ip}'

conn = psycopg.connect(db_connection)

In [7]:
stats_query = '''SELECT * FROM "yrden".lol_game_data
WHERE game_id in 
(select game_id
from "yrden".lol_game_data
where 1=1
and queue_id in (440, 700)
group by game_id
having count(game_id) = 10)
ORDER BY GAME_PATCH DESC, GAME_ID, TEAMID, CASE WHEN LANE = 'TOP' THEN 1 WHEN LANE = 'JUNGLE' THEN 2 WHEN LANE = 'MIDDLE' THEN 3 WHEN LANE = 'BOTTOM' THEN 4 ELSE 5 END
;'''

with conn.cursor() as cur:
    cur.execute(stats_query)
    col_names = [desc[0] for desc in cur.description]
    rows = cur.fetchall()
    df = pl.DataFrame(rows, schema=col_names, orient='row')

df.shape

(230, 59)

In [35]:
yrden_df = df.filter(pl.col('riot_puuid').is_in(constants.yrden_lol_team_puuids))

In [38]:
champ_df = yrden_df.select(pl.col('game_id'), pl.col('win'), pl.col('champion_id'), pl.col('riot_id'), pl.col('riot_puuid'))

In [85]:
pivot_df = champ_df.pivot('riot_id', index='game_id')
pivot_df.select(pl.col('game_id'), pl.col('win_Blue').alias('win'), pl.col('champion_id_Hypocritus'),
                pl.col('champion_id_Triggerman'), pl.col('champion_id_Blue'), pl.col('champion_id_YDN Rock Coaches'),
                pl.col('champion_id_wyzrdsnvrdie'))

game_id,win,champion_id_Hypocritus,champion_id_Triggerman,champion_id_Blue,champion_id_YDN Rock Coaches,champion_id_wyzrdsnvrdie
str,bool,i64,i64,i64,i64,i64
"""NA1_5206375709""",true,122,72,103,202,143
"""NA1_5206412184""",false,516,234,84,115,412
"""NA1_5207643890""",false,58,59,61,21,412
"""NA1_5207671213""",false,122,876,115,145,412
"""NA1_5205572005""",false,58,56,103,22,57
…,…,…,…,…,…,…
"""NA1_5168606154""",true,122,876,61,22,412
"""NA1_5170644901""",true,122,234,45,115,111
"""NA1_5170720153""",true,516,876,142,51,412


In [79]:
test_df = champ_df.with_columns(pl.col('champion_id').str.concat(delimiter=',').over('game_id').alias('concatentated_champ_ids'))

# test_df

# test_df = champ_df.with_columns(pl.col('concatenated_champ_ids'))
concat_df = test_df.select(pl.col('concatentated_champ_ids').str.split(',').list.to_struct(n_field_strategy='max_width')).unnest('concatentated_champ_ids')

pivoted_df = pl.concat([champ_df, concat_df], how='horizontal')

pivoted_df


C:\Users\Erica\AppData\Local\Temp\ipykernel_1168832\2540391368.py:6: UserWarning: `to_struct()` should be passed a list of field names to avoid query errors in subsequent operations (e.g. <struct operation> not supported for dtype Unknown)
  concat_df = test_df.select(pl.col('concatentated_champ_ids').str.split(',').list.to_struct(n_field_strategy='max_width')).unnest('concatentated_champ_ids')


game_id,win,champion_id,riot_id,riot_puuid,field_0,field_1,field_2,field_3,field_4
str,bool,i64,str,str,str,str,str,str,str
"""NA1_5206375709""",true,122,"""Hypocritus""","""Xhi01hd4YPU92OGkFKSsqEnTtVwTD_…","""122""","""72""","""103""","""202""","""143"""
"""NA1_5206375709""",true,72,"""Triggerman""","""0CpS1bKgKnH7IKuMVnpeOpyLFIj3J7…","""122""","""72""","""103""","""202""","""143"""
"""NA1_5206375709""",true,103,"""Blue""","""gw2MWsJzlQYIRr40fYEyB2RdJBixmb…","""122""","""72""","""103""","""202""","""143"""
"""NA1_5206375709""",true,202,"""YDN Rock Coaches""","""oFdCaHSdLE-umAd_AUTwskzbTOEPp-…","""122""","""72""","""103""","""202""","""143"""
"""NA1_5206375709""",true,143,"""wyzrdsnvrdie""","""nQd7Ylk472T9NtwuS-hhuLDcxoUg71…","""122""","""72""","""103""","""202""","""143"""
…,…,…,…,…,…,…,…,…,…
"""NA1_5177097514""",true,122,"""Hypocritus""","""Xhi01hd4YPU92OGkFKSsqEnTtVwTD_…","""122""","""113""","""163""","""222""","""111"""
"""NA1_5177097514""",true,113,"""Triggerman""","""0CpS1bKgKnH7IKuMVnpeOpyLFIj3J7…","""122""","""113""","""163""","""222""","""111"""
"""NA1_5177097514""",true,163,"""Blue""","""gw2MWsJzlQYIRr40fYEyB2RdJBixmb…","""122""","""113""","""163""","""222""","""111"""
